In [1]:
from sqlite_wrapper.sqlite_wrapper import SQLiteWrapper
import pandas as pd

In [2]:
db_wrapper = SQLiteWrapper("example.db", create=True)  # Set create to False after initial setup.

In [3]:
delete_table_sql = """
DROP TABLE IF EXISTS users;
"""

# Best practice would be to define all necessary constraints and indexes first
create_table_sql = """
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE
);
"""

# This creates a non-unique index on the 'name' column.
create_index_sql = """
CREATE INDEX idx_users_name ON users(name);
"""
db_wrapper.execute(delete_table_sql)
db_wrapper.execute(create_table_sql)
db_wrapper.execute(create_index_sql)

tables = db_wrapper.get_all_table_names()
print('\nTables are:')
tables

Statement executed successfully.
Statement executed successfully.
Statement executed successfully.

Tables are:


['users']

In [4]:
retrieved_df = db_wrapper.get_table('users')
print('\nCurrent contents of users:')
retrieved_df


Current contents of users:


,id,name,email


In [5]:
data_start = {
    'id': [1, 2, 3],
    'name': ["Alice", "Bob", "Charlie"],
    'email': ["alice@example.com", "bob@example.com", "charlie@example.com"]
}
data_start = pd.DataFrame(data_start)
data_start

,id,name,email
0,1,Alice,alice@example.com
1,2,Bob,bob@example.com
2,3,Charlie,charlie@example.com


In [6]:
data_append = {
    'id': [4, 5],
    'name': ["Diane", "Edward"],
    'email': ["diane@example.com", "edward@example.com"]
}
df_append = pd.DataFrame(data_append)
df_append

,id,name,email
0,4,Diane,diane@example.com
1,5,Edward,edward@example.com


In [7]:
print('\nExecuting initial data load (replace)')
db_wrapper.save_data(df=data_start, table_name='users', if_exists='replace')
db_wrapper.get_table('users')


Executing initial data load (replace)
Data saved to table 'users' successfully.


,id,name,email
0,1,Alice,alice@example.com
1,2,Bob,bob@example.com
2,3,Charlie,charlie@example.com


In [8]:
print('\nExecuting additional data load (append)')
db_wrapper.save_data(df=df_append, table_name='users', if_exists='append')
db_wrapper.get_table('users')



Executing additional data load (append)
Data saved to table 'users' successfully.


,id,name,email
0,1,Alice,alice@example.com
1,2,Bob,bob@example.com
2,3,Charlie,charlie@example.com
3,4,Diane,diane@example.com
4,5,Edward,edward@example.com


In [9]:
data_upsert = {
    'id': [2, 6],
    'name': ["Bob - Updated", "Fleur - New",],
    'email': ["bob_new@example.com", "fleur@example.com"]
}
df_upsert = pd.DataFrame(data_upsert)
df_upsert

,id,name,email
0,2,Bob - Updated,bob_new@example.com
1,6,Fleur - New,fleur@example.com


In [10]:
print('\nExecuting additional data load (upsert)')
db_wrapper.save_data(df=df_upsert, table_name='users', if_exists='upsert', unique_key='id')
db_wrapper.get_table('users')



Executing additional data load (upsert)
Data saved to table 'users' successfully.


,id,name,email
0,1,Alice,alice@example.com
1,2,Bob - Updated,bob_new@example.com
2,3,Charlie,charlie@example.com
3,4,Diane,diane@example.com
4,5,Edward,edward@example.com
5,6,Fleur - New,fleur@example.com


In [11]:
print('\nGetting the data and recreating the table definition with UUID id column')
df_users = db_wrapper.get_table('users')
df_users = df_users.drop(columns={'id'})

delete_table_sql = """
DROP TABLE IF EXISTS users;
"""

create_table_sql = """
CREATE TABLE IF NOT EXISTS users (
    id TEXT PRIMARY KEY, -- Store UUID as id
    name TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE
);
"""
db_wrapper.execute(delete_table_sql)
db_wrapper.execute(create_table_sql)

print('\nExecuting reload of data with uuid id column')
db_wrapper.save_data(
    df=df_users,
    table_name='users',
    if_exists='replace',
    auto_create=True,
    auto_add_id=True
    )

db_wrapper.get_table('users')


Getting the data and recreating the table definition with UUID id column
Statement executed successfully.
Statement executed successfully.

Executing reload of data with uuid id column
Data saved to table 'users' successfully.


,id,name,email
0,0190d8e3-f423-00d1-38bc-dcf44cd13bbf,Alice,alice@example.com
1,0190d8e3-f423-00d1-38bc-dcf5f53ca530,Bob - Updated,bob_new@example.com
2,0190d8e3-f423-00d1-38bc-dcf6cc09c9b3,Charlie,charlie@example.com
3,0190d8e3-f423-00d1-38bc-dcf7f5cfe8b0,Diane,diane@example.com
4,0190d8e3-f423-00d1-38bc-dcf864e3992f,Edward,edward@example.com
5,0190d8e3-f423-00d1-38bc-dcf94a329d51,Fleur - New,fleur@example.com


In [12]:
db_wrapper.flush()
print('Tables are:')
db_wrapper.get_all_table_names()

Database wiped successfully.
Tables are:


[]